In [ ]:
import pandas as pd # type: ignore
import os
import numpy as np # type: ignore

In [ ]:
# Directorio que contiene los archivos Excel
directory_path = "HOJAS_DE_INDEXACION"

# Obtener una lista de todos los archivos en el directorio
lista_archivos_excel = [f for f in os.listdir(directory_path) if f.endswith('.xlsx')]

# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre la lista de archivos y leer cada uno en un DataFrame
for archivo in lista_archivos_excel:
    ruta_completa = os.path.join(directory_path, archivo)
    if os.path.exists(ruta_completa):
        df = pd.read_excel(ruta_completa)
        lista_dataframes.append(df)
    else:
        print(f"El archivo {ruta_completa} no existe")

# Concatenar todos los DataFrames en uno solo
df_crudo = pd.concat(lista_dataframes, ignore_index=True)

# Mostrar el DataFrame final
print("DataFrame final:")
print(df_crudo.head())

# Guardar el DataFrame final en un archivo Excel (opcional)
df_crudo.to_excel("data_sin_limpiar.xlsx", index=False)

In [ ]:
# Información del dataframe
df_crudo.info()

In [ ]:
# Eliminar filas con valores nulos en la columna 'Num_problema'
df_sin_nulos = df_crudo.dropna(subset=['Num_problema'])

# Resetear los índices
df_sin_nulos.reset_index(drop=True, inplace=True)

# Revisando el DataFrame sin nulos en la Columna "Num_problema"
df_sin_nulos.head()

In [ ]:
# Información del dataframe
df_sin_nulos.info()

In [ ]:
# Revisando si hay preguntas String
conteo_string = df_sin_nulos['String'].value_counts(dropna=False)
conteo_string

In [ ]:
df_sin_nulos = df_sin_nulos.copy()

indices_a_eliminar = df_sin_nulos[(df_sin_nulos['String'] == 'B') | (df_sin_nulos['String'] == 'C')].index

df_sin_nulos.drop(indices_a_eliminar, inplace=True)

In [ ]:
df_sin_nulos.head()

In [ ]:
df_sin_nulos.info()

In [ ]:
# Obteniendo solo las columnas requeridas
columnas_no_requeridas = ['String', 'TER','OBSERVACIÓN','Unnamed: 11']
data = df_sin_nulos.drop(columns = columnas_no_requeridas)
data.head()

In [ ]:
# Nuevos nombres para todas las columnas
nuevos_nombres = [
    'NUM_PROBLEMA', 'COD_PROFESOR',
    'COD_CURSO', 'NO_TEMA',
    'NRO_SUBTEMA', 'NO_NIVEL',
    'CLAVE', 'TIPO_PREG'
    ]

# Cambiar los nombres de las columnas
data.columns = nuevos_nombres

# Mostrar los resultados
data.head()

In [ ]:
# Casteando columnas

# Identificar columnas que son de tipo float
columnas_float = data.select_dtypes(include=['float64']).columns

# Cambiar el tipo de dato de las columnas float a integer
data[columnas_float] = data[columnas_float].astype('Int64')

# Identificar columnas que son de tipo float
columnas_float = data.select_dtypes(include=['Int64']).columns

# Cambiar el tipo de dato de las columnas float a integer
data[columnas_float] = data[columnas_float].astype('object')

# revisando dataframe casteado
data.head(100)

In [ ]:
data.info()

In [ ]:
tema = '00'
subtema = '00'
nivel = '0'
clave = 'X'
tipo = 'X'

data.loc[data['NO_TEMA'].isna(), 'NO_TEMA'] = tema
data.loc[data['NRO_SUBTEMA'].isna(), 'NRO_SUBTEMA'] = subtema
data.loc[data['NO_NIVEL'].isna(), 'NO_NIVEL'] = nivel
data.loc[data['CLAVE'].isna(), 'CLAVE'] = clave
data.loc[data['TIPO_PREG'].isna(), 'TIPO_PREG'] = tipo
data.info()

In [ ]:
filtro = data['NRO_SUBTEMA'].isna()
data[filtro]

In [ ]:
data.info()

In [ ]:
# Definir una función para eliminar espacios de cualquier valor
def strip_spaces(value):
    if isinstance(value, str):
        return value.strip()
    elif isinstance(value, (int, float)):
        return value
    elif pd.isna(value):
        return value
    else:
        return str(value).strip()

In [ ]:
# Aplicar la función a cada elemento del DataFrame
data = data.map(strip_spaces)

# Mostrar el DataFrame resultante
data.head()

In [ ]:
data.info()

In [ ]:
# Casteando columnas
print()
# Identificar columnas que son de tipo float
columnas_float = data.select_dtypes(include=['float64']).columns

# Cambiar el tipo de dato de las columnas float a integer
data[columnas_float] = data[columnas_float].astype('Int64')

# revisando dataframe casteado
data.head()

In [ ]:
# Completando ceros en la columna 'NUM_PROBLEMA' hasta que tenga 6 dígitos

data['NUM_PROBLEMA'] = data['NUM_PROBLEMA'].astype(str).str.zfill(6)
data.head()

In [ ]:
# Concatenado las columnas 'NO_TEMA' y 'NRO_SUBTEMA'
data['COD_TEMARIO'] = data['NO_TEMA'].astype(str).str.zfill(2) + '.' + data['NRO_SUBTEMA'].astype(str).str.zfill(2)

# Mostrando DataFrame
data.head()

In [ ]:
# Creando una nueva columna con el código string del curso

diccionario_significados = {
    1: 'AR', 2: 'AL', 3: 'GM',
    4: 'TR', 5: 'RM', 6: 'RV',
    7: 'FI', 8: 'QU', 9: 'BI',
    10: 'AN', 11: 'LE', 12: 'LI',
    13: 'IN', 14: 'HP', 15: 'HU',
    16: 'GF', 17: 'EC', 18: 'CI',
    19: 'PS', 20: 'FL'
}

data['CURSO'] = data['COD_CURSO'].map(diccionario_significados)

data.head()

In [ ]:
# Creando la columna de reemplazo

data['REEMPLAZO'] = '@' + data[['NUM_PROBLEMA', 'CURSO','COD_TEMARIO', 'NO_NIVEL', 'CLAVE', 'TIPO_PREG']].astype('str').apply(" - ".join, axis=1)
data.head()

In [ ]:
sin_duplicados_reemplazo = data.drop_duplicates(subset = ['REEMPLAZO'], keep='first')

In [ ]:
sin_duplicados_reemplazo.info()

In [ ]:
sin_duplicados_reemplazo.duplicated('NUM_PROBLEMA').sum()

In [ ]:
# Identificar filas duplicadas en base a la columna 'columna1'
duplicados = sin_duplicados_reemplazo[sin_duplicados_reemplazo.duplicated(subset=['NUM_PROBLEMA'], keep=False)]

# Resetear los índices del DataFrame de duplicados
duplicados.reset_index(drop=True, inplace=True)

# Visualizando el nuevo data frame
duplicados.head()

In [ ]:
# Información del Dataframe duplicados
duplicados.info()

In [ ]:
duplicados_por_problema = duplicados['NUM_PROBLEMA'].value_counts()
duplicados_por_problema

In [ ]:
filtro = duplicados['NUM_PROBLEMA'] == '003321'
duplicados[filtro]

In [ ]:
columnas_fijas = ['NUM_PROBLEMA', 'REEMPLAZO']

matriz = duplicados[columnas_fijas]

matriz.head()

In [ ]:
# Asegurarse de que 'matriz' no es una copia antes de modificarla
matriz = matriz.copy()

# Crear un índice adicional para los valores de la columna 'Valor'
matriz.loc[:, 'idx'] = matriz.groupby('NUM_PROBLEMA').cumcount()

# Pivotar el DataFrame
matriz_duplicados = matriz.pivot(index='NUM_PROBLEMA', columns='idx', values='REEMPLAZO')

# Renombrar las columnas si es necesario
matriz_duplicados.columns = [f'DUPLICADO_{i+1}' for i in matriz_duplicados.columns]

# Resetear el índice para convertir el índice 'ID' en una columna
matriz_duplicados.reset_index(inplace=True)

# Mostrar el DataFrame pivotado
matriz_duplicados.head()

In [ ]:
# Definir una función para comparar los valores en una fila
def comparar_fila(row):
    if row.nunique() == 1:
        return 'IGUALES'
    else:
        return 'DIFERENTES'
    
# Seleccionar todas las columnas excepto 'ID' para la comparación
columnas_a_comparar = matriz_duplicados.columns.difference(['NUM_PROBLEMA'])

columnas_a_comparar

In [ ]:
# Aplicar la función a cada fila del DataFrame para las columnas de interés
matriz_duplicados['COMPARACION'] = matriz_duplicados[columnas_a_comparar].apply(comparar_fila, axis=1)

In [ ]:
matriz_duplicados.head()

In [ ]:
matriz_duplicados['COMPARACION'].value_counts()

In [ ]:
cruce = pd.merge(sin_duplicados_reemplazo, matriz_duplicados, on='NUM_PROBLEMA', how='outer')
cruce.head()

In [ ]:
proporcion = cruce['COMPARACION'].value_counts(dropna = False)
proporcion

In [ ]:
# Mask

diferentes = cruce['COMPARACION'] == 'DIFERENTES'

In [ ]:
duplicados_diferentes = cruce[diferentes]
duplicados_diferentes.head()

In [ ]:
duplicados_diferentes.info()

In [ ]:
duplicados_diferentes.to_excel("duplicados_diferentes.xlsx", index = False)

In [ ]:
unicos = ~ diferentes
registros_unicos = cruce[unicos]

registros_unicos.head()

In [ ]:
registros_unicos.info()

In [ ]:
valor_unico = 'UNICO'

registros_unicos.loc[:, 'COMPARACION'] = valor_unico

registros_unicos.head()

In [ ]:
registros_unicos.info()

In [ ]:
registros_unicos.to_excel("registros_unicos.xlsx", index = False)

In [ ]:
from extractor_final import extraer

extraer()

In [ ]:
# Cargar el archivo Excel
no_indexada = pd.read_excel("extraccion.xlsx")

# Remover caracteres no numéricos y espacios en blanco
no_indexada['PROBLEMA'] = no_indexada['PROBLEMA'].astype(str).str.strip().str.replace(r'\D', '', regex=True)

# Filtrar valores numéricos y conservar ceros iniciales
df_filtrada = no_indexada[no_indexada['PROBLEMA'].str.isdigit()].copy()

# Resetear el índice del DataFrame filtrado
df_filtrada.reset_index(drop=True, inplace=True)

# Guardar el DataFrame filtrado en un nuevo archivo Excel (opcional)
ruta_de_salida = "codigos_no_indexados.xlsx"  # Reemplaza con la ruta correcta para guardar el archivo
df_filtrada.to_excel(ruta_de_salida, index=False)

In [ ]:
df_filtrada.head()

In [ ]:
df_filtrada.info()

In [ ]:
df_filtrada.drop_duplicates(inplace = True)

df_filtrada.head()

In [ ]:
df_filtrada.info()

In [ ]:
seis_digitos = df_filtrada[df_filtrada['PROBLEMA'].str.match(r'^\d{6}$')]
seis_digitos.head()

In [ ]:
seis_digitos.info()

In [ ]:
seis_digitos = seis_digitos.copy()

seis_digitos.rename(columns={'PROBLEMA': 'NUM_PROBLEMA'}, inplace=True)

seis_digitos.head()

In [ ]:
full_data_6= pd.merge(registros_unicos, seis_digitos, on='NUM_PROBLEMA', how='outer')

In [ ]:
full_data_6.head(100)

In [ ]:
full_data_6.info()

In [ ]:
curso = input("ingrese el curso")

relleno = '@' + full_data_6['NUM_PROBLEMA'] + ' - ' + curso + ' - ' + '00.00' + ' - ' + '0' + ' - ' + 'X' + ' - ' + 'X'

full_data_6.loc[full_data_6['REEMPLAZO'].isna(), 'REEMPLAZO'] = relleno

full_data_6.head()

In [ ]:
full_data_6.reset_index(drop=True, inplace=True)
full_data_6.head()

In [ ]:
filtro = full_data_6['COMPARACION'] != 'UNICO'

data_con_seis = full_data_6[filtro]

data_con_seis.head()

In [ ]:
data_con_seis.info()

In [ ]:
filtro = df_filtrada['PROBLEMA'].str.match(r'^\d{6}$')

data_hasta_5 = df_filtrada[~ filtro]

data_hasta_5.head()

In [ ]:
data_hasta_5 = data_hasta_5.copy()

data_hasta_5['REEMPLAZO'] = '@' + data_hasta_5['PROBLEMA']

data_hasta_5.head()

In [ ]:
data_hasta_5.rename(columns={'PROBLEMA': 'NUM_PROBLEMA'}, inplace=True)

data_hasta_5.head()

In [ ]:
# Concatenar los DataFrames en base a las columnas 'ID' y 'NUM_PROBLEMA'
df_final = pd.merge(full_data_6, data_hasta_5, on=['NUM_PROBLEMA', 'REEMPLAZO'], how='outer')

# Resetear el índice del DataFrame concatenado
df_final.reset_index(drop=True, inplace=True)

# Mostrando del DataFrame
df_final.head(100)

In [ ]:
df_final.info()

In [ ]:
df_final.to_excel('data_final.xlsx', index = False)

In [ ]:
input("DALE GOOO!!!!")

## PRIMER REEMPLAZO

In [ ]:
from pathlib import Path  # core python module
import win32com.client  # type: ignore # pip install pywin32

# Path settings
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
input_dir = current_dir / "INPUT"
output_dir = current_dir / "OUTPUT"
output_dir.mkdir(parents=True, exist_ok=True)


# Creando el diccionario
diccionario = full_data_6[['NUM_PROBLEMA', 'REEMPLAZO']].set_index('NUM_PROBLEMA')['REEMPLAZO'].to_dict()


word_app = win32com.client.DispatchEx("Word.Application")
word_app.Visible = False
word_app.DisplayAlerts = False


for doc_file in Path(input_dir).rglob("*.doc*"):

    for word, replacement in diccionario.items():
        
        find_str = word
        replace_with = replacement 
        wd_replace = 2  # 2=replace all occurences, 1=replace one occurence, 0=replace no occurences
        wd_find_wrap = 1  # 2=ask to continue, 1=continue search, 0=end if search range is reached

        # Open each document and replace string
        word_app.Documents.Open(str(doc_file))
        # API documentation: https://learn.microsoft.com/en-us/office/vba/api/word.find.execute
        word_app.Selection.Find.Execute(
            FindText=find_str,
            ReplaceWith=replace_with,
            Replace=wd_replace,
            Forward=True,
            MatchCase=True,
            MatchWholeWord=False,
            MatchWildcards=True,
            MatchSoundsLike=False,
            MatchAllWordForms=False,
            Wrap=wd_find_wrap,
            Format=True,
        )

    # Save the new file
    output_path = output_dir / f"{doc_file.stem}_replaced{doc_file.suffix}"
    word_app.ActiveDocument.SaveAs(str(output_path))
    word_app.ActiveDocument.Close(SaveChanges=False)
word_app.Application.Quit()

## SEGUNDO REEMPLAZO

from pathlib import Path
import win32com.client  # type: ignore # pip install pywin32

# Path settings
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
input_dir = current_dir / "OUTPUT"
output_dir = current_dir / "REEMPLAZO_TOTAL"
output_dir.mkdir(parents=True, exist_ok=True)

# Creando el diccionario
diccionario = data_mala_final[['NUM_PROBLEMA', 'REEMPLAZO']].set_index('NUM_PROBLEMA')['REEMPLAZO'].to_dict()

# Iniciar la aplicación de Word
word_app = win32com.client.DispatchEx("Word.Application")
word_app.Visible = False
word_app.DisplayAlerts = False

for doc_file in Path(input_dir).rglob("*.doc*"):
    # Abrir cada documento
    doc = word_app.Documents.Open(str(doc_file))
    
    for word, replacement in diccionario.items():
        # Condición para omitir números que comiencen con "@"
        if not word.startswith('@'):
            find_str = word
            replace_with = replacement 
            wd_replace = 2  # 2=replace all occurences, 1=replace one occurence, 0=replace no occurences
            wd_find_wrap = 1  # 2=ask to continue, 1=continue search, 0=end if search range is reached

            # Reemplazar texto en el documento
            word_app.Selection.Find.Execute(
                FindText=find_str,
                ReplaceWith=replace_with,
                Replace=wd_replace,
                Forward=True,
                MatchCase=True,
                MatchWholeWord=False,
                MatchWildcards=True,
                MatchSoundsLike=False,
                MatchAllWordForms=False,
                Wrap=wd_find_wrap,
                Format=True,
            )

    # Guardar el nuevo archivo
    output_path = output_dir / f"{doc_file.stem}_final{doc_file.suffix}"
    doc.SaveAs(str(output_path))
    doc.Close(SaveChanges=False)

# Cerrar la aplicación de Word
word_app.Application.Quit()